In [1]:
import numpy as np
import pandas as pd

In [2]:
# Import Dataset
data_og = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/00373/drug_consumption.data', sep=",", names=['Age','Gender','Education','Country','Ethnicity','Nscore','Escore','Oscore','Ascore','Cscore','Impulsive','SS','Alcohol','Amphet','Amyl','Benzos','Caff','Cannabis','Choc','Coke','Crack','Ecstasy','Heroin','Ketamine','Legalh','LSD','Meth','Mushrooms','Nicotine','Semer','VSA'])
data_og.head()

data = pd.read_csv('drugs.csv')

In [3]:
d = data_og
d.drop(d[d['Semer'] != 'CL0'].index, inplace = True)
d = d.drop('Semer', 1)
cannabis = d['Cannabis']
d = d.drop(['Cannabis'], 1)

C:\Users\Zack\AppData\Local\Temp/ipykernel_20980/3832084421.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  d = d.drop('Semer', 1)
C:\Users\Zack\AppData\Local\Temp/ipykernel_20980/3832084421.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  d = d.drop(['Cannabis'], 1)


One-hot encoding for qualitative variables

In [4]:

drug_labels = ['Alcohol','Amphet','Amyl','Benzos','Caff','Choc','Coke','Crack','Ecstasy','Heroin','Ketamine','Legalh','LSD','Meth','Mushrooms','Nicotine','VSA']
personality_labels = ['Nscore','Escore','Oscore','Ascore','Cscore','Impulsive','SS']
demographic_labels = ['Age','Gender','Education','Country','Ethnicity']
sch_labels = ['Sch1','Sch2','Sch3','Sch4','SchNA']
for drug in drug_labels:
    d[drug] = np.where(d[drug] == 'CL0', 0, 1)
    # not a user = 0, user = 1
    
    
x = d

#for p in personality_labels:
    #x[p] = x[p].astype('category').cat.codes.to_numpy()
x = pd.get_dummies(x, columns=(personality_labels))
    
#for d in demographic_labels:
#    x[d] = x[d].astype('category').cat.codes.to_numpy()
x = pd.get_dummies(x, columns=(demographic_labels))

One-hot encoding of classes (CL0 - CL6) for target

In [5]:
targets = cannabis.astype('category').cat.codes.to_numpy()
y = np.zeros((7,len(targets)))
for m in range(len(targets)):
    y[targets[m], m] = 1


y = y.T

#print(x.shape,y.shape)
#for col in x.columns:
#    print(col)

Split data into train and test, create and fit classifier

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [7]:
from sklearn.neural_network import MLPClassifier

print(X_train.shape[1], 'input nodes, ', y_train.shape[1], 'output nodes')
clf = MLPClassifier(solver='sgd', activation='relu',
                    hidden_layer_sizes=(16,), random_state=1, max_iter = 500)
clf.fit(X_train, y_train)

277 input nodes,  7 output nodes


C:\Users\Zack\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(16,), max_iter=500, random_state=1,
              solver='sgd')

Using .score results in 25.5% accuracy. However, this is a  multi-class classifier, so some of its predictions have multiple output neurons active and some have none. Using argmax of each probability prediction is a better indicator for our output with only a single class, which brings the accuracy up to 45.4%.

In [8]:
from sklearn.metrics import accuracy_score, confusion_matrix

accuracy = clf.score(X_test, y_test)
print("Accuracy: ",accuracy) 


Accuracy:  0.2553191489361702


In [9]:
probs = clf.predict_log_proba(X_test)

print("Confusion Matrix: \n", confusion_matrix(y_test.argmax(axis=1), probs.argmax(axis=1)))
print("\nAccuracy: ", correct/probs.shape[0])


Confusion Matrix: 
 [[72  3  8  1  0  0  4]
 [25 11  7  2  0  0  3]
 [11  0 14  5  0  0 13]
 [ 2  3  8  2  0  0 18]
 [ 3  1  2  1  0  0 26]
 [ 0  0  4  3  0  0 38]
 [ 1  3  9  1  0  0 72]]

Accuracy:  0.45478723404255317


----------

Multiclass, only drugs

For similar reasons to above, using argmax increases the model's accuracy from 18.0% all the way up to 83.2%. Stark increase in accuracy when compared to including demographic and personality data. 

In [81]:
x = d
for l in demographic_labels:
    x = x.drop(l, 1)
    
for l in personality_labels:
    x = x.drop(l, 1)
    
    
targets = cannabis.astype('category').cat.codes.to_numpy()
y = np.zeros((7,len(targets)))
for m in range(len(targets)):
    y[targets[m], m] = 1


y = y.T

print(x.shape, y.shape)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

clf = MLPClassifier(solver='sgd', activation='relu',
                    hidden_layer_sizes=(12,), random_state=1, max_iter = 500)
clf.fit(X_train, y_train)

accuracy = clf.score(X_test, y_test)
print("Accuracy: ",accuracy) 

probs = clf.predict_log_proba(X_test)

print("Confusion Matrix: \n", confusion_matrix(y_test.argmax(axis=1), probs.argmax(axis=1)))
print("\nAccuracy: ", correct/probs.shape[0])

C:\Users\Zack\AppData\Local\Temp/ipykernel_20980/2714676579.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  x = x.drop(l, 1)
C:\Users\Zack\AppData\Local\Temp/ipykernel_20980/2714676579.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  x = x.drop(l, 1)


(1877, 17) (1877, 7)
Accuracy:  0.18085106382978725
Confusion Matrix: 
 [[79  0  5  0  0  0  2]
 [25  0  9  0  0  0  7]
 [18  0  5  0  0  0 21]
 [ 6  0  7  0  0  0 35]
 [ 6  0  2  0  0  0 21]
 [ 2  0  3  0  0  0 34]
 [ 7  0  4  0  0  0 78]]

Accuracy:  0.8324468085106383


C:\Users\Zack\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


-----------

multiclass, only drug schedules

argmax brings it up from 13.8% to 83.2%. 

In [82]:
x = data

for l in demographic_labels:
    x = x.drop(l, 1)
    
for l in personality_labels:
    x = x.drop(l, 1)
    
for l in drug_labels:
    x = x.drop(l, 1)
    
x = x.drop(['Cannabis'], 1)
    
targets = cannabis.astype('category').cat.codes.to_numpy()
y = np.zeros((7,len(targets)))
for m in range(len(targets)):
    y[targets[m], m] = 1


y = y.T

print(x.shape, y.shape)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

clf = MLPClassifier(solver='sgd', activation='relu',
                    hidden_layer_sizes=(5,), random_state=1, max_iter = 500)
clf.fit(X_train, y_train)

accuracy = clf.score(X_test, y_test)
print("Accuracy: ",accuracy) 

probs = clf.predict_log_proba(X_test)

print("Confusion Matrix: \n", confusion_matrix(y_test.argmax(axis=1), probs.argmax(axis=1)))
print("\nAccuracy: ", correct/probs.shape[0])

C:\Users\Zack\AppData\Local\Temp/ipykernel_20980/1446547358.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  x = x.drop(l, 1)
C:\Users\Zack\AppData\Local\Temp/ipykernel_20980/1446547358.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  x = x.drop(l, 1)
C:\Users\Zack\AppData\Local\Temp/ipykernel_20980/1446547358.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  x = x.drop(l, 1)
C:\Users\Zack\AppData\Local\Temp/ipykernel_20980/1446547358.py:12: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  x = x.drop(['Cannabis'], 1)


(1877, 5) (1877, 7)
Accuracy:  0.13829787234042554
Confusion Matrix: 
 [[69  0  0  0  0  0  0]
 [30  0  0  0  0  0 25]
 [24  0  0  0  0  0 34]
 [ 8  0  0  0  0  0 27]
 [ 4  0  0  0  0  0 23]
 [ 6  0  0  0  0  0 36]
 [ 4  0  0  0  0  0 86]]

Accuracy:  0.8324468085106383


Both multiclass models have a tendency to only guess C0 and C6, so it is not as robust as the 83% number may seem.

 ----------------------
 
 Binary classification, all input variables

In [10]:
newtargets = np.where(targets == 0, 0, 1)

y = np.zeros((2,len(newtargets)))
for m in range(len(newtargets)):
    y[newtargets[m], m] = 1
y = y.T

Training a new model for binary classification output

In [11]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

clf = MLPClassifier(solver='sgd', activation='relu',
                    hidden_layer_sizes=(16,), random_state=1, max_iter = 500)

clf.fit(X_train, y_train)

C:\Users\Zack\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(16,), max_iter=500, random_state=1,
              solver='sgd')

Just as previously done, argmax of each probability prediction was used, which brings the accuracy up slightly from 85.6% to 86.2%. This jump is expected to be only slight due to the greatly reduced number of output classifications available, and so the backwards propogation doesn't reward a non-classification as much in this scenario.

In [12]:
accuracy = clf.score(X_test, y_test)
print("Accuracy: ",accuracy) 

Accuracy:  0.8563829787234043


In [13]:
probs = clf.predict_log_proba(X_test)

print("Confusion Matrix: \n", confusion_matrix(y_test.argmax(axis=1), probs.argmax(axis=1)))
print("\nAccuracy: ", correct/probs.shape[0])

Confusion Matrix: 
 [[ 58  24]
 [ 28 266]]

Accuracy:  0.8617021276595744


------------------

Binary classification of the output using only drugs as input variables (no demographics or personality types). This ended up being the most accurate prediction model. 

In [14]:
x = d
for l in demographic_labels:
    x = x.drop(l, 1)
    
for l in personality_labels:
    x = x.drop(l, 1)


C:\Users\Zack\AppData\Local\Temp/ipykernel_20980/4018020744.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  x = x.drop(l, 1)
C:\Users\Zack\AppData\Local\Temp/ipykernel_20980/4018020744.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  x = x.drop(l, 1)


In [26]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

clf = MLPClassifier(solver='sgd', activation='relu',
                    hidden_layer_sizes=(12,), random_state=1, max_iter = 500)

clf.fit(X_train, y_train)

C:\Users\Zack\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(12,), max_iter=500, random_state=1,
              solver='sgd')

Argmax of each probability prediction was used, which brings the accuracy up slightly from 89.3% to 89.6%.

In [27]:
accuracy = clf.score(X_test, y_test)
print("Accuracy: ",accuracy) 

Accuracy:  0.8936170212765957


In [28]:
probs = clf.predict_log_proba(X_test)

print("Confusion Matrix: \n", confusion_matrix(y_test.argmax(axis=1), probs.argmax(axis=1)))
print("\nAccuracy: ", correct/probs.shape[0])

Confusion Matrix: 
 [[ 51  27]
 [ 12 286]]

Accuracy:  0.8962765957446809


-------------------------

Binary classification of the output using only drug classifications as input variables (no demographics, personality types, or usage of individual drugs)


In [34]:
x = data

for l in demographic_labels:
    x = x.drop(l, 1)
    
for l in personality_labels:
    x = x.drop(l, 1)
    
for l in drug_labels:
    x = x.drop(l, 1)
    
x = x.drop(['Cannabis'], 1)

C:\Users\Zack\AppData\Local\Temp/ipykernel_20980/886407707.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  x = x.drop(l, 1)
C:\Users\Zack\AppData\Local\Temp/ipykernel_20980/886407707.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  x = x.drop(l, 1)
C:\Users\Zack\AppData\Local\Temp/ipykernel_20980/886407707.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  x = x.drop(l, 1)
C:\Users\Zack\AppData\Local\Temp/ipykernel_20980/886407707.py:12: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  x = x.drop(['Cannabis'], 1)


In [35]:
x.columns

Index(['Sch1', 'Sch2', 'Sch3', 'Sch4', 'SchNA'], dtype='object')

In [47]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

clf = MLPClassifier(solver='sgd', activation='relu',
                    hidden_layer_sizes=(5,), random_state=1, max_iter = 500)

clf.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(5,), max_iter=500, random_state=1,
              solver='sgd')

Argmax of each probability prediction was used, which brings the accuracy up slightly from 81.4% to 83.2%.

In [48]:
accuracy = clf.score(X_test, y_test)
print("Accuracy: ",accuracy) 

taccuracy = clf.score(X_train, y_train)
print("Training Accuracy: ",taccuracy) 

Accuracy:  0.8138297872340425
Training Accuracy:  0.8181212524983345


In [75]:
probs = clf.predict_log_proba(X_test)

print("Confusion Matrix: \n", confusion_matrix(y_test.argmax(axis=1), probs.argmax(axis=1)))
print("\nAccuracy: ", correct/probs.shape[0])

Confusion Matrix: 
 [[ 64  19]
 [ 44 249]]

Accuracy:  0.8324468085106383


In [42]:
print(X_test.shape, y_test.shape)

(376, 5) (376, 2)


In [79]:
clf.coefs_

[array([[-6.01928455e-02,  6.64220694e-02, -4.28120477e-01,
         -2.61525595e-01, -3.81764588e-01, -4.42569607e-01,
         -1.79962064e-01,  3.21498779e-01,  9.17126628e-02,
         -2.76389025e-02, -1.59960505e-01,  7.27329287e-01],
        [-2.89955785e-01,  2.21044473e-01, -4.29934217e-01,
          5.11629352e-02, -1.49544556e-01,  1.01784073e-02,
         -3.29017474e-01,  8.07515869e-02,  3.29630296e-01,
          3.66388982e-01, -1.74262944e-01, -4.45372528e-01],
        [ 3.88878762e-01,  2.52424006e-01, -3.77484149e-01,
         -4.27038765e-01, -3.06481635e-01,  2.64549138e-01,
         -3.86246453e-01, -2.27280414e-01,  7.60850223e-01,
          7.96082051e-04,  1.53673126e-01, -2.85869438e-01],
        [ 1.65708285e-01,  1.81479329e-01, -4.41709269e-01,
          1.40594930e-01,  3.82481575e-01,  2.04768735e-01,
         -1.67680853e-01,  1.56742183e-01, -1.23894048e-01,
         -6.95581096e-02,  3.06534137e-01, -2.10483502e-01],
        [-1.83102958e-01, -4.6708453